In [14]:
from pingouin import ancova, read_dataset
df = read_dataset('ancova')

In [15]:
df.sample(5)

,Scores,Income,BMI,Method
14,33,49.8,18,B
13,40,84.8,18,B
1,39,104.6,20,A
6,8,20.0,29,A
23,21,80.5,21,C


In [16]:
model = ancova(data=df, dv="Scores", covar="Income", between="Method")
model

,Source,SS,DF,F,p-unc,np2
0,Method,571.029883,3,3.336482,0.031940,0.244077
1,Income,1678.352687,1,29.419438,0.000006,0.486920
2,Residual,1768.522313,31,NaN,NaN,NaN


In [17]:
ancova(data=df, dv="Scores", covar=["Income", "BMI"], between="Method", effsize="n2")

,Source,SS,DF,F,p-unc,n2
0,Method,552.284043,3,3.232550,0.036113,0.141802
1,Income,1573.952434,1,27.637304,0.000011,0.404121
2,BMI,60.013656,1,1.053790,0.312842,0.015409
3,Residual,1708.508657,30,NaN,NaN,NaN


<table style="margin: 10px; padding: 10px; border-collapse: collapse; border: 2px solid black;">
  <tr>
    <th style="padding: 10px; border: 2px solid black;">Aspect</th>
    <th style="padding: 10px; border: 2px solid black;">1-Way ANOVA</th>
    <th style="padding: 10px; border: 2px solid black;">2-Way ANOVA</th>
    <th style="padding: 10px; border: 2px solid black;">MANOVA</th>
  </tr>
  <tr>
    <td style="padding: 10px; border: 2px solid black;">Purpose</td>
    <td style="padding: 10px; border: 2px solid black;">Test differences across 1 factor.</td>
    <td style="padding: 10px; border: 2px solid black;">Test differences across 2 factors and their interaction.</td>
    <td style="padding: 10px; border: 2px solid black;">Test differences across multiple dependent variables.</td>
  </tr>
  <tr>
    <td style="padding: 10px; border: 2px solid black;">Independent Vars</td>
    <td style="padding: 10px; border: 2px solid black;">1 factor</td>
    <td style="padding: 10px; border: 2px solid black;">2 factors</td>
    <td style="padding: 10px; border: 2px solid black;">1 or more factors</td>
  </tr>
  <tr>
    <td style="padding: 10px; border: 2px solid black;">Dependent Vars</td>
    <td style="padding: 10px; border: 2px solid black;">1 continuous</td>
    <td style="padding: 10px; border: 2px solid black;">1 continuous</td>
    <td style="padding: 10px; border: 2px solid black;">2 or more continuous</td>
  </tr>
  <tr>
    <td style="padding: 10px; border: 2px solid black;">Interaction</td>
    <td style="padding: 10px; border: 2px solid black;">Not considered</td>
    <td style="padding: 10px; border: 2px solid black;">Considered</td>
    <td style="padding: 10px; border: 2px solid black;">Not considered (interaction within DVs possible separately)</td>
  </tr>
  <tr>
    <td style="padding: 10px; border: 2px solid black;">Example</td>
    <td style="padding: 10px; border: 2px solid black;">Effect of fertilizer on growth.</td>
    <td style="padding: 10px; border: 2px solid black;">Effect of fertilizer and watering on growth.</td>
    <td style="padding: 10px; border: 2px solid black;">Effect of fertilizer on growth and chlorophyll content.</td>
  </tr>
</table>


In [37]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import shapiro, levene, f_oneway
from statsmodels.formula.api import ols
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.multivariate.manova import MANOVA
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Example data generation
def generate_data():
    np.random.seed(42)
    
    # One-Way ANOVA data
    group_a = np.random.normal(50, 10, 30)
    group_b = np.random.normal(55, 10, 30)
    group_c = np.random.normal(60, 10, 30)
    anova_data = pd.DataFrame({
        'score': np.concatenate([group_a, group_b, group_c]),
        'group': ['A'] * 30 + ['B'] * 30 + ['C'] * 30
    })

    # Two-Way ANOVA data
    two_way_data = pd.DataFrame({
        'score': np.random.normal(50, 10, 100),
        'factor_1': np.random.choice(['Low', 'High'], size=100),
        'factor_2': np.random.choice(['Male', 'Female'], size=100)
    })

    # MANOVA data
    manova_data = pd.DataFrame({
        'dependent_1': np.random.normal(50, 10, 50),
        'dependent_2': np.random.normal(60, 15, 50),
        'group': np.random.choice(['Control', 'Treatment'], size=50)
    })

    # ANCOVA data
    ancova_data = pd.DataFrame({
        'dependent': np.random.normal(50, 10, 100),
        'independent': np.random.choice(['A', 'B'], size=100),
        'covariate': np.random.normal(5, 2, 100)
    })

    return anova_data, two_way_data, manova_data, ancova_data

anova_data, two_way_data, manova_data, ancova_data = generate_data()



In [34]:
anova_data.sample(5)

,score,group
38,63.225449,B
89,54.982430,C
60,51.607825,C
4,43.759254,A
11,11.203606,A


In [26]:
two_way_data.head()

,score,factor_1,factor_2
0,50.970775,Low,Male
1,59.686450,Low,Male
2,42.979469,High,Male
3,46.723379,Low,Female
4,46.078918,High,Female


In [23]:
manova_data.head()

,dependent_1,dependent_2,group
0,40.184913,44.909739,Control
1,54.621035,41.787171,Control
2,51.990597,77.371663,Control
3,43.997831,71.874940,Treatment
4,50.698021,69.361797,Treatment


In [24]:
ancova_data.head()

,dependent,independent,covariate
0,57.419630,A,3.870886
1,64.766586,B,2.995907
2,57.002430,B,5.559515
3,48.677006,A,-0.481910
4,49.263889,A,4.082761


In [38]:
import statsmodels.api as sm
# 1. One-Way ANOVA
def one_way_anova(data):
    print("One-Way ANOVA")
    model = ols('score ~ C(group)', data=data).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    # print(anova_table)

    # Diagnostics
    residuals = model.resid

    # Normality check
    stat, p = shapiro(residuals)
    print(f"Shapiro-Wilk Test p-value: {p}")
    
    if p < 0.05:
        print("Residuals are not normally distributed. Consider transformations.")

    # Homogeneity of variance check
    stat, p = levene(data[data['group'] == 'A']['score'],
                     data[data['group'] == 'B']['score'],
                     data[data['group'] == 'C']['score'])
    print(f"Levene's Test p-value: {p}")

    if p < 0.05:
        print("Variance is not equal across groups. Consider transformations.")

    return anova_table

# Run One-Way ANOVA
one_way_anova(anova_data)

One-Way ANOVA
Shapiro-Wilk Test p-value: 0.89481520652771
Levene's Test p-value: 0.8626638812717601


,sum_sq,df,F,PR(>F)
C(group),2165.964313,2.0,12.209526,0.000021
Residual,7716.880356,87.0,NaN,NaN


In [50]:
def two_way_anova(data):
    print("Two-Way ANOVA")
    
    # Fit an OLS model with interaction between two factors
    model = ols('score ~ C(factor_1) * C(factor_2)', data=data).fit()
    
    # Perform ANOVA and print the table
    anova_table = sm.stats.anova_lm(model, typ=2)
    # print(anova_table)

    # Diagnostics: Residuals analysis
    residuals = model.resid
    stat, p = shapiro(residuals)
    print(f"Shapiro-Wilk Test p-value: {p}")
    if p < 0.05:
        print("Residuals are not normally distributed. Consider transformations.")

    # Homogeneity of variance: Levene's test
    stat, p = levene(data[data['factor_1'] == 'Low']['score'],
                     data[data['factor_1'] == 'High']['score'])
    print(f"Levene's Test p-value: {p}")
    if p < 0.05:
        print("Variance is not equal across factors. Consider transformations.")

    return anova_table

# Run Two-Way ANOVA
two_way_anova(two_way_data)

Two-Way ANOVA
Shapiro-Wilk Test p-value: 0.13747896254062653
Levene's Test p-value: 0.15256312784582918


,sum_sq,df,F,PR(>F)
C(factor_1),0.146761,1.0,0.001581,0.968369
C(factor_2),0.786457,1.0,0.008471,0.926861
C(factor_1):C(factor_2),10.957857,1.0,0.118022,0.731941
Residual,8913.171845,96.0,NaN,NaN


In [48]:
manova_data.sample(5)

,dependent_1,dependent_2,group
15,43.773005,51.544132,Treatment
2,51.990597,77.371663,Control
13,55.883172,55.179212,Treatment
43,46.897332,60.188886,Control
7,56.621307,46.541184,Control


In [65]:
manova_data.sample(5)

,dependent_1,dependent_2,group
8,65.860168,61.137068,Control
19,58.496021,52.395852,Control
36,47.823188,67.790198,Treatment
23,53.072995,38.888043,Treatment
10,71.330334,74.626796,Treatment


In [95]:
# from statsmodels.stats.multivariate import BoxM
import pingouin as pg
def manova(data):
    print("MANOVA")
    
    # Fit MANOVA model with multiple dependent variables
    manova_model = MANOVA.from_formula('dependent_1 + dependent_2 ~ group', data=data)
    
    # Print MANOVA test results
    print(manova_model.mv_test())

    # Assumptions: Multivariate normality
    dependent_vars = data[['dependent_1', 'dependent_2']]
    stat, p = shapiro(dependent_vars)
    print(f"Shapiro-Wilk Test for Multivariate Normality p-value: {p}")
    if p < 0.05:
        print("Dependent variables are not multivariate normal.")

    # Homogeneity of covariance matrices: Box's M test
    box_m_result = pg.box_m(data,['dependent_1', 'dependent_2'], 'group')
    print(box_m_result)
    print(f"Box's M Test p-value: {box_m_result.pval}")
    if box_m_result.pval[0] < 0.05:
        print("Covariance matrices are not equal. Consider transformations.")
    # return box_m_result
# Run MANOVA
manova(manova_data)

# 4. ANCOVA

MANOVA
                  Multivariate linear model
                                                              
--------------------------------------------------------------
       Intercept         Value  Num DF  Den DF F Value  Pr > F
--------------------------------------------------------------
          Wilks' lambda  0.0349 2.0000 47.0000 649.7714 0.0000
         Pillai's trace  0.9651 2.0000 47.0000 649.7714 0.0000
 Hotelling-Lawley trace 27.6498 2.0000 47.0000 649.7714 0.0000
    Roy's greatest root 27.6498 2.0000 47.0000 649.7714 0.0000
--------------------------------------------------------------
                                                              
--------------------------------------------------------------
           group          Value  Num DF  Den DF F Value Pr > F
--------------------------------------------------------------
            Wilks' lambda 0.9284 2.0000 47.0000  1.8127 0.1744
           Pillai's trace 0.0716 2.0000 47.0000  1.8127 0.1744
   H

In [87]:
box_m_result.pval

box    0.247657
Name: pval, dtype: float64

In [96]:
# 4. ANCOVA
# Function to perform ANCOVA and check assumptions
def ancova(data):
    print("ANCOVA")
    
    # Fit an OLS model with group and covariate as predictors
    model = ols('dependent ~ C(independent) + covariate', data=data).fit()
    
    # Perform ANCOVA and print the table
    anova_table = sm.stats.anova_lm(model, typ=2)
    print(anova_table)

    # Check for homogeneity of regression slopes by testing interaction
    interaction_model = ols('dependent ~ C(independent) * covariate', data=data).fit()
    interaction_table = sm.stats.anova_lm(interaction_model, typ=2)
    print("Interaction Table:")
    print(interaction_table)

    # Diagnostics: Residual analysis
    residuals = model.resid
    stat, p = shapiro(residuals)
    print(f"Shapiro-Wilk Test p-value: {p}")
    if p < 0.05:
        print("Residuals are not normally distributed. Consider transformations.")

# Run ANCOVA
ancova(ancova_data)


ANCOVA
                      sum_sq    df         F    PR(>F)
C(independent)      5.512417   1.0  0.052725  0.818872
covariate         105.978720   1.0  1.013653  0.316534
Residual        10141.477880  97.0       NaN       NaN
Interaction Table:
                                sum_sq    df         F    PR(>F)
C(independent)                5.512417   1.0  0.052225  0.819721
covariate                   105.978720   1.0  1.004055  0.318850
C(independent):covariate      8.612232   1.0  0.081593  0.775765
Residual                  10132.865647  96.0       NaN       NaN
Shapiro-Wilk Test p-value: 0.8003330230712891
